In [2]:
import pandas as pd

In [3]:
def get_all():
    base = pd.read_excel(r'/home/gustavo/Documentos/ListedOff/data/Base.xlsx', engine='openpyxl')
    return base

In [33]:
def handling_base(base):
    base['Instrument'] = base['Ticker'].apply(lambda x: " ".join(x.split()[:-1]))
    base['Instrument'] = [t[:-1] for t in base['Instrument']]
    base = x.groupby(['YellowKey','Instrument']).first().reset_index()
    base = base[['YellowKey', 'Instrument', 'Maturity Label', 'Style', 'Maturity', 'Underlying']]
    base.to_csv(r'/home/gustavo/Documentos/ListedOff/data/Base_aux.csv', index=False)

def read_base_aux():
    base = pd.read_csv(r'/home/gustavo/Documentos/ListedOff/data/Base_aux.csv')
    return base

In [5]:
def tickers_to_booking():
    tickers = pd.read_excel(r'/home/gustavo/Documentos/ListedOff/data/Tickers.xlsx', header=None, engine='openpyxl')
    tickers = tickers.rename(columns={0:'Ticker_Full'})
    return tickers

In [8]:
def get_ticker_yellow_key():
    df = tickers_to_booking()
    df['Ticker'] = df['Ticker_Full'].apply(lambda x: " ".join(x.split()[:-1]))
    df['YellowKey'] = [y.split()[-1] for y in df['Ticker_Full']]
    df['Instrument'] = df['Ticker'].apply(lambda x: " ".join(x.split()[:-1]))
    df['Instrument'] = [t[:-1] for t in df['Instrument']]
    return df

In [26]:
def check_ticker():
    base = get_all()
    tickers = get_ticker_yellow_key()
    base_aux = read_base_aux()
    tickers['New'] = ''
    msg = []
    
    for t,i,j in zip(tickers.Ticker, tickers.Instrument, tickers.index):
        if base.Ticker.str.contains(t).any():
            tickers.drop(tickers[tickers['Ticker'] == t].index, inplace=True)
            msg.append("Ticker " + t + " já consta na base")
            tickers['New'][j] = False
        elif base_aux.Instrument.str.contains(i).any():
            tickers['New'][j] = False
            pass            
        else:
            msg.append("O ticker " + t + " é novo")
            tickers['New'][j] = True
    
    return msg, tickers      

In [ ]:
def tickers_to_upload():
    msg, tickers = check_ticker()
    
    if not tickers:
        return "Não há tickers para cadastrar"
    
    
    

In [ ]:
def construct_dataframe(base_aux, instrument, yellow_key, ticker):
    df = pd.DataFrame()
    df['Ticker'] = ''
    if base_aux.Instrument.str.contains(instrument):
        df = base_aux.loc[base_aux['Instrument'] == instrument]
        if df.YellowKey.str.contains(yellow_key):
             df['Ticker'] = ticker
    return df
    
    

In [46]:
msg,tickers = check_ticker()
strike = [t.split()[-1]for t in tickers.Ticker]
payout =  list(tickers['Ticker'].apply(lambda x: " ".join(x.split()[:-1])))
payout = [p[-1] for p in payout]
payout = ["Call" if p == "C" else "Put" for p in payout]
payout

['Call', 'Put', 'Call']

In [37]:
read_base_aux()

,YellowKey,Instrument,Maturity Label,Style,Maturity,Underlying
0,Comdty,SCH1,H1 2605,A,2021-05-26,SCH1
1,Index,S J1,J1 2607,A,2021-07-26,S M1


In [28]:
x,y = check_ticker()
for i in x:
    print(i)
y

Ticker SCH1C 3000 já consta na base
O ticker SCJ1C 3000 é novo


,Ticker_Full,Ticker,YellowKey,Instrument,New
0,S J1C 3000 Index,S J1C 3000,Index,S J1,False
1,SCH1P 3200 Comdty,SCH1P 3200,Comdty,SCH1,False
3,SCJ1C 3000 Comdty,SCJ1C 3000,Comdty,SCJ1,True
